In [109]:
#demonstration notebook for range indexing in VV

In [110]:
import numpy as np
import pyart
import math
#import pandas as pd

In [111]:
  #Load the range variable
    # if your range changes every elevation then you 
    # need to use the slower index_VV
r2fg = 2125
num_gates = 1200
gate_spacing_m = 250

grange = r2fg + np.arange(0, num_gates) * gate_spacing_m
elev_range = grange #this is pretty normal for radars the gatespacing is constant in the 
#entire volume

In [112]:
VV_elevation = 0.0 
data_elevation = 45.0 #the demo is easier to see at 45 degrees of elevation

vv_data = np.zeros(num_gates)

radar_data = np.arange(0,num_gates) #our input data is just an increasing series


In [113]:
# now we convert the range at 45 degrees into a range at the vv_elevation of zero degrees
elev_diff = data_elevation - VV_elevation
conv_range = elev_range * math.cos(math.radians(elev_diff))

In [114]:
#now we convert the above range into an index on the VV elevation
conv_index = np.rint((conv_range-r2fg)/gate_spacing_m).astype(int)

In [115]:
for i in range(0,10):
    print(i, " radar: ", radar_data[i], "conv index: ", conv_index[i])

0  radar:  0 conv index:  -2
1  radar:  1 conv index:  -2
2  radar:  2 conv index:  -1
3  radar:  3 conv index:  0
4  radar:  4 conv index:  0
5  radar:  5 conv index:  1
6  radar:  6 conv index:  2
7  radar:  7 conv index:  2
8  radar:  8 conv index:  3
9  radar:  9 conv index:  4


In [116]:
#the negatives are bad: remove them

var_index = conv_index


In [117]:
for i in range(0,10):
    print(i, " radar: ", radar_data[i], "conv index: ", var_index[i])

0  radar:  0 conv index:  -2
1  radar:  1 conv index:  -2
2  radar:  2 conv index:  -1
3  radar:  3 conv index:  0
4  radar:  4 conv index:  0
5  radar:  5 conv index:  1
6  radar:  6 conv index:  2
7  radar:  7 conv index:  2
8  radar:  8 conv index:  3
9  radar:  9 conv index:  4


In [118]:
print("Range:", conv_range[50:75])

Range: [10341.43667485 10518.21337015 10694.99006545 10871.76676074
 11048.54345604 11225.32015134 11402.09684663 11578.87354193
 11755.65023723 11932.42693252 12109.20362782 12285.98032312
 12462.75701841 12639.53371371 12816.31040901 12993.0871043
 13169.8637996  13346.6404949  13523.41719019 13700.19388549
 13876.97058079 14053.74727608 14230.52397138 14407.30066668
 14584.07736197]


In [119]:
print("Index:", var_index[50:75])

Index: [33 34 34 35 36 36 37 38 39 39 40 41 41 42 43 43 44 45 46 46 47 48 48 49
 50]


In [120]:
#note all the repeated indexs! We only want one data per index!


In [121]:
# we want to compare var_index at i with var_index at i-1

#shift the conv_index one element to the right
shift_var_index = np.insert(var_index,0,-100, axis=0)

#append a value onto the end of var_index to make them match in size
var_index = np.append(var_index,var_index[-1])

In [122]:
print(var_index[0:10], shift_var_index[0:10])

[-2 -2 -1  0  0  1  2  2  3  4] [-100   -2   -2   -1    0    0    1    2    2    3]


In [123]:

#using where identify the locations we want to keep
non_dups = np.where(((var_index != shift_var_index) & (var_index>=0)), True, False)
#non_dups = non_dups[np.where(var_index >0, True, False)]


In [124]:
#chop non_dups back down:
non_dups = non_dups[0:len(non_dups)-1]


In [125]:
for i in range(50,75):
    print(i, " radar: ", radar_data[i], "var index: ", var_index[i], " Keep? ", non_dups[i])

50  radar:  50 var index:  33  Keep?  True
51  radar:  51 var index:  34  Keep?  True
52  radar:  52 var index:  34  Keep?  False
53  radar:  53 var index:  35  Keep?  True
54  radar:  54 var index:  36  Keep?  True
55  radar:  55 var index:  36  Keep?  False
56  radar:  56 var index:  37  Keep?  True
57  radar:  57 var index:  38  Keep?  True
58  radar:  58 var index:  39  Keep?  True
59  radar:  59 var index:  39  Keep?  False
60  radar:  60 var index:  40  Keep?  True
61  radar:  61 var index:  41  Keep?  True
62  radar:  62 var index:  41  Keep?  False
63  radar:  63 var index:  42  Keep?  True
64  radar:  64 var index:  43  Keep?  True
65  radar:  65 var index:  43  Keep?  False
66  radar:  66 var index:  44  Keep?  True
67  radar:  67 var index:  45  Keep?  True
68  radar:  68 var index:  46  Keep?  True
69  radar:  69 var index:  46  Keep?  False
70  radar:  70 var index:  47  Keep?  True
71  radar:  71 var index:  48  Keep?  True
72  radar:  72 var index:  48  Keep?  False
73  

In [126]:
#find the index's of the locations we want to keep
vv_data = radar_data[np.where(non_dups==True)]
print (vv_data[35:50])

[53 54 56 57 58 60 61 63 64 66 67 68 70 71 73]


In [127]:
#note the radar values of 52, 55, 59, 65, 69 are missing and "false" above

In [128]:
for i in range(0,10):
    print(i, " radar: ", vv_data[i])

0  radar:  3
1  radar:  5
2  radar:  6
3  radar:  8
4  radar:  9
5  radar:  10
6  radar:  12
7  radar:  13
8  radar:  15
9  radar:  16


In [129]:
#if you look above "3" is the fisrt value with index 0